In [1]:
import warnings
warnings.filterwarnings("ignore", message=".*`torch.cuda.amp.autocast.*")
import torch
import numpy as np
import random
import PIL.Image
import matplotlib.pyplot as plt
from types import SimpleNamespace
from datasets import load_dataset
from torchvision.transforms.v2 import (
    Compose, Resize, RandomCrop, CenterCrop,
    PILToTensor, ToPILImage, CutMix )
from fastprogress.fastprogress import master_bar, progress_bar
from IPython.display import display, HTML

In [2]:
torch.set_float32_matmul_precision('high')
device = "cuda:2"
dataset_train = load_dataset('danjacobellis/inet_cheat_288_webp',split='train')
dataset_valid = load_dataset('danjacobellis/inet_cheat_288_webp',split='validation')

Resolving data files:   0%|          | 0/61 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/61 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/60 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/61 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/61 [00:00<?, ?it/s]

In [3]:
config = SimpleNamespace()
# Training and optimizer config
config.batch_size = 128
config.steps_per_epoch = dataset_train.num_rows//config.batch_size
config.grad_accum_steps = 1
config.max_lr = (config.batch_size/128)*6e-4
config.min_lr = config.max_lr/100
config.plot_update = 128
config.epochs = 100
config.lr_scale = 1
config.lr_offset = 0.25
config.lr_pow = 6
config.weight_decay = 0.
config.num_workers = 12
config.image_size = 288
config.crop_size = 256
config.cutmix_probability = 1.
config.cutmix_alpha = 1.

# model config
config.channels = 3
config.J = 4
config.embed_dim = 512
config.classifier_num_classes = 1000
config.checkpoint = False

In [4]:
from pytorch_wavelets import DWTForward
from tft.transforms import WPT2D
from tft.models import TFTClassifier

wt = DWTForward(J=1, mode='periodization', wave='bior4.4')
wpt = WPT2D(wt).to(device)
model = TFTClassifier(config,wpt).to(device)

for name, module in model.named_children():
    print(f"{sum(p.numel() for p in module.parameters())/1e6} \t {name}")

if config.checkpoint:
    checkpoint = torch.load(config.checkpoint, map_location="cpu", weights_only=False)
    model.load_state_dict(checkpoint['model'])

0.0 	 wpt
22.503955 	 ascan
0.263684 	 pool
0.513 	 classifier


In [5]:
x = torch.randn(128,3,256,256).to(device)
with torch.no_grad():
    y = model(x)

In [6]:
y.shape

torch.Size([128, 1000])